In [1]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

/usr/bin/bash: /home/lucas/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/bash)
--2022-12-04 13:03:22--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  11.3MB/s    in 15s     

2022-12-04 13:03:39 (5.21 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

/usr/bin/bash: /home/lucas/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/bash)


In [2]:
import tensorflow as tf
from tensorflow import keras

dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 100006 files belonging to 1 classes.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2022-12-04 13:05:23.217838: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 13:05:25.320193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3077 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [4]:
from tensorflow.keras.layers import TextVectorization

sequence_length = 100
vocab_size = 15000

text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_sequence_length=sequence_length,
    output_mode="int",
)
text_vectorization.adapt(dataset)

In [5]:
def prepare_dataset(text_batch):
    vectorized_sequence=text_vectorization(text_batch)
    x = vectorized_sequence[:, :-1]
    y = vectorized_sequence[:, 1:]
    return x, y

im_dataset = dataset.map(prepare_dataset, num_parallel_calls=4)


In [7]:
from tensorflow.keras import layers
embed_dim =256
latent_dim=1024
num_heads=2

inputs = keras.Input(shape=(None,), dtype="int64")
x=PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x=TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs=layers.Dense(vocab_size, activation="softmax")(x)
model=keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")


NameError: name 'PositionalEmbedding' is not defined